In [1]:
# importing the required libraries
import math
import random
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.svm import SVC
from sklearn import metrics
import matplotlib.pyplot as plt
from sklearn.preprocessing import scale
from sklearn.model_selection import KFold
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, hinge_loss

letters = pd.read_csv("/content/sample_data/letter-recognition.csv")

In [2]:
#print(letters.shape)
print(letters.columns)

Index(['lettr', 'x-box', 'y-box', 'width', 'high', 'onpix', 'x-bar', 'y-bar',
       'x2bar', 'y2bar', 'xybar', 'x2ybr', 'xy2br', 'x-ege', 'xegvy', 'y-ege',
       'yegvx'],
      dtype='object')


In [3]:
x=letters['lettr'].unique()

In [4]:
round(letters.drop('lettr', axis=1).mean(), 2)

x-box    4.02
y-box    7.04
width    5.12
high     5.37
onpix    3.51
x-bar    6.90
y-bar    7.50
x2bar    4.63
y2bar    5.18
xybar    8.28
x2ybr    6.45
xy2br    7.93
x-ege    3.05
xegvy    8.34
y-ege    3.69
yegvx    7.80
dtype: float64

In [5]:
X = letters.drop("lettr", axis = 1)
y = letters['lettr']

In [6]:
X_scaled = scale(X)
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size = 0.3, random_state = 101)
print(X_train.shape)
# X_train.describe()
print(type(X_scaled))
print(y_train.shape)
print(X_test.shape)

(14000, 16)
<class 'numpy.ndarray'>
(14000,)
(6000, 16)


In [7]:
def eval(m,cnt_neg,cnt_plus,zt):
  if m%2 == 0:
    if zt[16] == 'A':
        cnt_plus += 1;
    else:
        cnt_neg += 1

In [8]:
def markov_samp(X_train, Y_train, k = 10, q = 1.2):
    samp = []
    for iter in range(k):
        a=np.vstack(X_train)
        b=np.vstack(Y_train.to_numpy())
        Init_sampl = np.concatenate((a, b), axis = 1)
        Dtr = random.sample(list(Init_sampl), int(2000))
        m = len(Dtr)
        print("M : " , m)

        cnt_neg = 0
        cnt_plus = 0

        index = np.random.choice(len(Dtr), 1, replace=False)  
        Dtra = np.array(Dtr)
        print(type(Dtra))
        zt = Dtra[index][0]

        eval(m,cnt_neg,cnt_plus,zt)

        model_linear = SVC(kernel='linear')
        model_linear.fit(X_train, Y_train)
        y_zt = model_linear.predict(np.array(zt)[0:16].reshape(1, -1))
        fxy1 = 1
        if not y_zt==zt[16]:
            fxy1 = 2 
        l_zt = math.exp(0-fxy1)
        Pd = 0
        Pdd = 0
        while(cnt_plus + cnt_neg < m ):
            zstar = Dtra[np.random.choice(len(Dtr), 1, replace=False)][0]
            ystar = model_linear.predict(np.array(zstar)[ 0:16].reshape(1, -1)) 
            fxy = 1
            if not ystar==zstar[16]:
                fxy = 2 
            lzstar = math.exp(0-fxy)
            P = math.exp(l_zt-lzstar)
            
            yt = zt[16]
            zt = zstar

            if P == 1:
                if zt[16] == yt:
                    r = random.uniform(0.001, 1.0)
                    if r <= P:
                        samp.append(zstar) 
                else:
                    if yt=='A':
                      ct=1
                    else: ct=-1

                    if ystar=='A':
                      cst=1
                    else: cst=-1

                    Pd = math.exp(ct*fxy1-cst*fxy)
                    r = random.uniform(0.001, 1.0)
                    if r <= Pd:
                        samp.append(zstar) 

            if len(samp) == k:
                Pdd = q*P
                samp.append(zstar) 
                
            ztp1 = zstar
            
            if yt == 'A':
                cnt_plus += 1
            else:
                cnt_neg += 1

            if P > 1 or Pd > 1 or Pdd > 1:
                samp.append(zstar)
    return samp

In [9]:
num_samp = np.array(markov_samp(X_train, y_train))
print(num_samp.shape)
X_train = num_samp[:, 0:16]
y_train = num_samp[:, 16]
print(X_train.shape)
print(y_train.shape)

M :  2000
<class 'numpy.ndarray'>
M :  2000
<class 'numpy.ndarray'>
M :  2000
<class 'numpy.ndarray'>
M :  2000
<class 'numpy.ndarray'>
M :  2000
<class 'numpy.ndarray'>
M :  2000
<class 'numpy.ndarray'>
M :  2000
<class 'numpy.ndarray'>
M :  2000
<class 'numpy.ndarray'>
M :  2000
<class 'numpy.ndarray'>
M :  2000
<class 'numpy.ndarray'>
(18662, 17)
(18662, 16)
(18662,)


In [10]:
Y_train = []
for i in y_train:
    Y_train.append(ord(i))

In [11]:
print(X_test.shape)

(6000, 16)


In [12]:
Y_test = []
for i in y_test:
    Y_test.append(ord(i))

In [13]:
# X_train.describe()

In [14]:
class KernelManager:
  def __init__(self, xTrain, yTrain, xTest, yTest):
    self.xTrain = xTrain
    self.yTrain = yTrain
    self.xTest = xTest
    self.yTest = yTest

  def linearKernel(self):
    modelLinear = SVC(kernel='linear')
    initialText = "The accuracy using linear kernel : "
    return modelLinear, initialText

  def rbfKernel(self):
    modelLinear = SVC(kernel='rbf')
    initialText = "The accuracy using rbf kernel : "
    return modelLinear, initialText

  def polyKernel(self):
    modelLinear = SVC(kernel='poly')
    initialText = "The accuracy using poly kernel : "
    return modelLinear, initialText

  def findAccuracy(self, modelLinear, initialText):
    modelLinear.fit(self.xTrain, self.yTrain)
    y_pred = modelLinear.predict(self.xTest)
    accuracy = metrics.accuracy_score(y_true=self.yTest, y_pred=y_pred)
    print(initialText, accuracy, "\n") 
    return accuracy

In [15]:
manager = KernelManager(X_train, Y_train, X_test, Y_test)
acc_array = []

In [16]:
model_linear, txt = manager.linearKernel()
acc_array.append(str(1 - float(manager.findAccuracy(model_linear, txt)))[:4])

The accuracy using linear kernel :  0.8456666666666667 



In [17]:
model_linear, txt = manager.rbfKernel()
acc_array.append(str(1 - float(manager.findAccuracy(model_linear, txt)))[:4])

The accuracy using rbf kernel :  0.9348333333333333 



In [18]:
model_linear, txt = manager.polyKernel()
acc_array.append(str(1 - float(manager.findAccuracy(model_linear, txt)))[:4])

The accuracy using poly kernel :  0.8885 



In [19]:
# acc_array.append("NaN")
# acc_array


In [20]:
table = pd.DataFrame(columns=['Kernel', 'KPCA', 'SVDD', 'OCSVM', 'OCSSVM', 'OCSSVM with SMO', 'MS_SVM'])
table['Kernel'] = ['Linear', 'RBF', 'Intersection']
table['KPCA'] = ['0.02', '0.05', '0.18']
table['SVDD'] = ['0.09', '0.07', '0.01']
table['OCSVM'] = ['0.01', '0.14', '0.04']
table['OCSSVM'] = ['0.07', '0.09', '0.26']
table['OCSSVM with SMO'] = ['0.04', '0.04', '0.22']
table['MS_SVM'] = acc_array
table

,Kernel,KPCA,SVDD,OCSVM,OCSSVM,OCSSVM with SMO,MS_SVM
0,Linear,0.02,0.09,0.01,0.07,0.04,0.15
1,RBF,0.05,0.07,0.14,0.09,0.04,0.06
2,Intersection,0.18,0.01,0.04,0.26,0.22,0.11


In [22]:
# acc_array